In [2]:
import numpy as np 
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

import sklearn
%precision 3

import requests, zipfile
import io

!pipenv install xlrd

In [16]:
trans = pd.read_excel('Online Retail.xlsx', sheet_name='Online Retail')
trans.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [17]:
# InvoiceNo開頭的第一個字作cancel_flg追加
trans['cancel_flg'] =trans.InvoiceNo.map(lambda x:str(x)[0])

# cancel_flg分群統計
trans.groupby('cancel_flg').size()

cancel_flg
5    532618
A         3
C      9288
dtype: int64

In [21]:
trans = trans[(trans.cancel_flg == '5') & (trans.CustomerID.notnull())]

## 規則關聯

In [26]:
trans['StockCode'].value_counts().head()

85123A    2035
22423     1724
85099B    1618
84879     1408
47566     1397
Name: StockCode, dtype: int64

## 規則關連的支持度

In [40]:
# 全部的InvoiceNo 抽出為 trans_all
trans_all = set(trans.InvoiceNo)

# '85123A' set 'trans_a'
trans_a = set(trans[trans['StockCode'] == '85123A'].InvoiceNo)
print(len(trans_a))

# '85099B' set 'trans_b'
trans_b = set(trans[trans['StockCode'] == '85099B'].InvoiceNo)
print(len(trans_b))

# 將'85123A'與'85099B' 資料置於 'trans_ab'
trans_ab = trans_a & trans_b
print(len(trans_ab))

1978
1600
252


In [38]:
print('含有兩個商品的購物籃數量:{}'.format(len(trans_ab)))
print('含有兩個商品的購物籃佔全體比例:{:.3f}'.format(len(trans_ab)/len(trans_all)))

含有兩個商品的購物籃數量:252
含有兩個商品的購物籃佔全體比例:0.014


In [39]:
print('含有85123A的購物籃數量:{}'.format(len(trans_a)))
print('含有85123A的購物籃佔全體比例:{:.3f}'.format(len(trans_a)/len(trans_all)))

含有85123A的購物籃數量:1978
含有85123A的購物籃佔全體比例:0.107


# 可信度

### 買了85123A 也買了 85099B 之規則下的可信度

In [41]:
print('可信度:{:.3f}'.format(len(trans_ab)/len(trans_a)))

可信度:0.127


### 買了 85099B  也買了 85123A 之規則下的可信度

In [43]:
print('可信度:{:.3f}'.format(len(trans_ab)/len(trans_b)))

可信度:0.158


# 增益值

In [47]:
support_b = len(trans_b) / len(trans_all)

confidence = len(trans_ab) / len(trans_a)

lift = confidence / support_b
lift

1.476